In [1]:
from selenium import webdriver
import os
import time
import sys
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlretrieve
import re

In [2]:
search=input('검색어를 입력하세요: ')
url="https://www.musinsa.com/"

검색어를 입력하세요:  닥터마틴


In [3]:
browser=webdriver.Chrome(executable_path="chromedriver.exe") #chromedriver 실행-윈도우
#browser = webdriver.Chrome('/Users/malangcongdduck/Downloads/chromedriver')#chromedriver 실행-맥
browser.get(url) #검색

In [4]:
#소스코드가 있는 경로에 '검색어' 폴더가 없으면 만들어준다
if not os.path.exists(search):
    os.mkdir(search)

In [5]:
search_box = browser.find_element_by_xpath('//*[@class="search head-search-inp keyword-dec"]') #검색창 
search_box.send_keys(search)
time.sleep(1)

In [6]:
# 전체 소스 가져오기
full_html = browser.page_source
soup = BeautifulSoup(full_html, 'html.parser')

In [7]:
brand_recommend=[] #추천 상품리스트의 브랜드명 리스트
item_recommend=[] #추천 상품리스트의 상품명 리스트
recommend_list = soup.find_all('dd', class_='recommendProduct listItem ui-good-listiem clearfix') #추천 리스트 전체 가져오기

for item in recommend_list:
    brand_name=item.find('p', class_='brand').text
    item_name=item.find('p', class_='product ellipsis').text
    brand_recommend.append(brand_name)
    item_recommend.append(item_name)
    


In [8]:
#데이터 프레임 생성
recommend_df = pd.DataFrame([ x for x in zip(brand_recommend,item_recommend)])
recommend_df.columns=['브랜드','상품명']
print(recommend_df)

#원하는 상품 선택해서 검색
search_num=int(input('원하는 상품의 번호를 입력하세요: '))
browser.find_element_by_xpath('//*[@class="recommendProduct-list list"]/dd['+str(search_num+1)+']/div/a').click()
time.sleep(2)

      브랜드                               상품명
0  [닥터마틴]             테리 블랙 브란도 샌들 23521001
1  [닥터마틴]                그리폰 블랙 샌들 15695001
2  [닥터마틴]                나틸라 블랙 샌들 24641001
3  [닥터마틴]               마일즈 블랙 슬리퍼 23523001
4  [닥터마틴]                                애봇
5  [닥터마틴]        1461 3홀 클롯 블랙 스무스 27153001
6  [닥터마틴]            블레어 블랙 브란도 샌들 24191001
7  [닥터마틴]    1B99 버지니아 14홀 부츠 블랙 / 11820008
8  [닥터마틴]                단테 - 블랙 / 16736001
9  [닥터마틴]               올슨 블랙 샌들 / 26561001


원하는 상품의 번호를 입력하세요:  0


In [9]:
#상세 페이지 소스 가져오기
full_html = browser.page_source
soup = BeautifulSoup(full_html, 'html.parser')

In [10]:
result_name=soup.find('span', class_='product_title').find('em').text #상품명
item_image=soup.find('div', class_='product-img').find('img')
image_src=item_image['src']
image_src=image_src.replace('//', 'https://')
urlretrieve( image_src , f'{search}/{result_name}.jpg')

('닥터마틴/테리 블랙 브란도 샌들 23521001.jpg', <http.client.HTTPMessage at 0x1dc4f88d0a0>)

In [20]:
category=soup.find('p', class_='item_categories').text
category=category.split()
category_name=category[0]
detail_category_name=category[2]
brand_name=category[3]
brand_name=brand_name.replace("(","")
brand_name=brand_name.replace(")","")

print(category_name, detail_category_name, brand_name)

닥터마틴
신발 샌들 닥터마틴


In [21]:
category_review=[]

#리뷰 종류
estimate_style = soup.find('span', id='estimate_style')
estimate_photo = soup.find('span', id='estimate_photo')
estimate_goods = soup.find('span', id='estimate_goods')

# 여러자리 숫자 단위를 category_name에서 리뷰개수 추출
category_review.append(re.findall("\d+", str(estimate_style)))
category_review.append(re.findall("\d+", str(estimate_photo)))
category_review.append(re.findall("\d+", str(estimate_goods)))

for i in category_review:
    if len(i) != 1:
        i[0]=i[0]+i[1]
        del i[1]
        
print(category_review)
category_review = sum(category_review, [])
category_review = list(map(int,category_review))
print(category_review)


[['53'], ['133'], ['204']]
[53, 133, 204]


In [22]:
# 전체 소스 가져오기
full_html = browser.current_url
full_html = full_html.split('/')

#상품번호와 리뷰 페이지이름
product = full_html[-1]
print(product)

1334440


In [23]:
score_style=[]
text_style=[]
count = 0
page=1 #2
review_index = 0

#스타일 후기 리뷰 크롤링
review_page = 'style'
browser.find_element_by_xpath('//*[@class="snb"]/li['+str(1)+']/h4/span').click()
page_num=browser.find_element_by_xpath("//div[@class='wrap review_list_detail style_review']/div/div[@class='nslist_bottom']/div").text
page_num = re.findall("\d+", page_num)
page_num = list(map(int,page_num)) #page_num[0]은 마지막 페이지를 나타냄
page = page_num[1]
end_page = page_num[0]

while(True):
    print(page, end_page)
    time.sleep(3) 
    content_list_style = browser.find_elements_by_css_selector('div.wrap.review_list_detail.style_review > div.galleryList.gallery_store.review-wrap > div.nslist_post.area-gallery-post.review-list') #스타일후기
    
    for i in range(1, len(content_list_style)+1):
        count+=1
        
        #각 요소 가져오기
        tmp_score_tag = browser.find_element_by_xpath("//div[@class='wrap review_list_detail style_review']/div/div["+(str(i))+"]/div[2]/div/div[@class='review-size']/div")
        tmp_name = tmp_score_tag.get_attribute('innerHTML') #태그의 클래스 이름 가져오기
        tmp_score = re.findall("\d+", tmp_name)
        tmp_score = int(tmp_score[0])
        #print(tmp_score)

        tmp_text = browser.find_element_by_xpath("//div[@class='wrap review_list_detail style_review']/div/div["+(str(i))+"]/div[2]/div/div[@class='pContent']") #태그
        tmp_text = tmp_text.text
        #print(tmp_text)
        
        score_style.append(tmp_score)
        text_style.append(tmp_text)

        print(f"총 {category_review[review_index]} 건 중 {count} 번째 리뷰 데이터를 수집합니다====================================")
        print('1) 별점:', tmp_score)
        print('2) 리뷰내용:', tmp_text)
        print('\n')

        # 만약 현재 글 수가 입력건수에 도달하면 루프 종료
        if(count == category_review[review_index]):
            break

    if(count == category_review[review_index]):
        break

    # 아직 입력건수에 도달하지 않았다면 다음 페이지를 열고 루프 계속
    else:
        page=page+1
        if page > end_page:
            print('삭제된 리뷰를 제외한 모든 리뷰를 가져왔습니다.')
            break
        browser.execute_script(f'viewEstimateByPage("{product}","0","{page}","{review_page}","","","N","new")')
        browser.implicitly_wait(5)

1 6
총 53 건 중 1 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 10
2) 리뷰내용: 예뻐요 잠옷이긴한데 에뻐오~~~~~~ 사이즈듀 딱이에여


총 53 건 중 2 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 10
2) 리뷰내용: 닥터마틴이라해서 불편하고 무거운줄알았는데 진짜편함


총 53 건 중 3 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 10
2) 리뷰내용: 구두부터 그냥 운동화까지 전부다 250 무조건 신는데 이건 무조건 한치수 작게 사라고 해서 240으로 주문했어요. 다른 닥터마틴은 전부 무거운데 테리는 진짜 가벼웠어요!!


총 53 건 중 4 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 10
2) 리뷰내용: 왼쪽은 복숭아뼈가 부딪히지않고, 오른쪽은 복숭아뼈에 닿아서 조금 까졌어요. 익숙해지면 더 잘 씻고 다닐거같아요


총 53 건 중 5 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 6
2) 리뷰내용: Wow 너무 예뻐요 ~~ 활용도도 높을 것 같고 밑창 땜에 키가 커보이는 효과까지 ^^ 근데 단 한 가지 단점 발 뒤꿈치가 너무 불편해요 ^^ 길가다가 신발 벗고 맨발로 갈 뻔 했어요.. 길들이는 기간이 필요하다는 점.. 명심하고 사시면 좋을 것 같네요.. 역시 예쁜 값을 하는 군


총 53 건 중 6 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 10
2) 리뷰내용: 이쁜데 맨발에 신으면 뒤에 까져용 ㅠ
헝ㅠㅡㅠㅠ사이즈는 큰거갗애여!-!


총 53 건 중 7 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 1

In [26]:
score_photo=[]
text_photo=[]
count = 0
page=1 #2
review_index = 1

#상품 사진 후기 리뷰 크롤링
review_page = 'photo'
browser.find_element_by_xpath('//*[@class="snb"]/li['+str(2)+']/h4/span').click()
page_num=browser.find_element_by_xpath("//div[@class='wrap review_list_detail photo_review']/div/div[@class='nslist_bottom']/div").text
page_num = re.findall("\d+", page_num)
page_num = list(map(int,page_num)) #page_num[0]은 마지막 페이지를 나타냄
page = page_num[1]
end_page = page_num[0]

while(True):
    time.sleep(5) 
    content_list_photo = browser.find_elements_by_css_selector('div.wrap.review_list_detail.photo_review > div.galleryList.gallery_store.review-wrap > div.nslist_post.area-gallery-post.review-list') #스타일후기
    
    for i in range(1, len(content_list_photo)+1):
        count+=1
        
        #각 요소 가져오기
        tmp_score_tag = browser.find_element_by_xpath("//div[@class='wrap review_list_detail photo_review']/div/div["+(str(i))+"]/div[2]/div/div[@class='review-size']/div")
        tmp_name = tmp_score_tag.get_attribute('innerHTML') #태그의 클래스 이름 가져오기
        tmp_score = re.findall("\d+", tmp_name)
        tmp_score = int(tmp_score[0])
        #print(tmp_score)

        tmp_text = browser.find_element_by_xpath("//div[@class='wrap review_list_detail photo_review']/div/div["+(str(i))+"]/div[2]/div/div[@class='pContent']") #태그
        tmp_text = tmp_text.text
        #print(tmp_text)
        
        score_photo.append(tmp_score)
        text_photo.append(tmp_text)

        print(f"총 {category_review[review_index]} 건 중 {count} 번째 리뷰 데이터를 수집합니다====================================")
        print('1) 별점:', tmp_score)
        print('2) 리뷰내용:', tmp_text)
        print('\n')

        # 만약 현재 글 수가 입력건수에 도달하면 루프 종료
        if(count == category_review[review_index]):
            break

    if(count == category_review[review_index]):
        break

    # 아직 입력건수에 도달하지 않았다면 다음 페이지를 열고 루프 계속
    else:
        page=page+1
        if page > end_page:
            print('삭제된 리뷰를 제외한 모든 리뷰를 가져왔습니다.')
            break
        browser.execute_script(f'viewEstimateByPage("{product}","0","{page}","{review_page}","","","N","new")')
        browser.implicitly_wait(5)

IndexError: list index out of range

In [32]:
score_goods=[]
text_goods=[]
count = 0
page=1 #2
review_index = 2

#일반 후기 리뷰 크롤링
review_page = 'goods'
browser.find_element_by_xpath('//*[@class="snb"]/li['+str(3)+']/h4/span').click()
page_num=browser.find_element_by_xpath("//div[@class='wrap review_list_detail estimate_review']/div/div[@class='nslist_bottom']/div").text
page_num = re.findall("\d+", page_num)
page_num = list(map(int,page_num)) #page_num[0]은 마지막 페이지를 나타냄
page = page_num[1]
end_page = page_num[0]

while(True):
    time.sleep(3) 
    content_list_goods = browser.find_elements_by_css_selector('div.wrap.review_list_detail.estimate_review > div.galleryList.gallery_store.review-wrap > div.nslist_post.area-gallery-post.review-list') #일반 후기
    
    for i in range(1, len(content_list_goods)+1):
        count+=1
        
        #각 요소 가져오기
        tmp_score_tag = browser.find_element_by_xpath("//div[@class='wrap review_list_detail estimate_review']/div/div["+(str(i))+"]/div[2]/div/div[@class='review-size']/div")
        tmp_name = tmp_score_tag.get_attribute('innerHTML') #태그의 클래스 이름 가져오기
        tmp_score = re.findall("\d+", tmp_name)
        tmp_score = int(tmp_score[0])
        #print(tmp_score)

        tmp_text = browser.find_element_by_xpath("//div[@class='wrap review_list_detail estimate_review']/div/div["+(str(i))+"]/div[2]/div/div[@class='pContent']") #태그
        tmp_text = tmp_text.text
        #print(tmp_text)
        
        score_goods.append(tmp_score)
        text_goods.append(tmp_text)

        print(f"총 {category_review[review_index]} 건 중 {count} 번째 리뷰 데이터를 수집합니다====================================")
        print('1) 별점:', tmp_score)
        print('2) 리뷰내용:', tmp_text)
        print('\n')

        # 만약 현재 글 수가 입력건수에 도달하면 루프 종료
        if(count == category_review[review_index]):
            break

    if(count == category_review[review_index]):
        break

    # 아직 입력건수에 도달하지 않았다면 다음 페이지를 열고 루프 계속
    else:
        page=page+1
        if page > end_page:
            print('삭제된 리뷰를 제외한 모든 리뷰를 가져왔습니다.')
            break
        browser.execute_script(f'viewEstimateByPage("{product}","0","{page}","{review_page}","","","N","new")')
        browser.implicitly_wait(5)

총 606 건 중 1 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 10
2) 리뷰내용: 자주 신을 것 같습니다~ 아무 옷이랑 다 잘 어울려요


총 606 건 중 2 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 10
2) 리뷰내용: 어우 왜 이제야 샀을까 하는? 운동화 중 기본템으로 사두면 후회없는 아이템입니다. 그나마 단점은 너무 많이 신는다는거?


총 606 건 중 3 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 10
2) 리뷰내용: 매치하기 쉬운 옷이라서 너무 좋아요 따뜻합니다


총 606 건 중 4 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 8
2) 리뷰내용: 아버지 선물로 괜찮네요
착화감도 괜찮으시다고 합니다


총 606 건 중 5 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 10
2) 리뷰내용: 테일윈드랑 데이브레이크 고민했는데 잘산거같아요


총 606 건 중 6 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 10
2) 리뷰내용: 누나한테 사줬는대 이쁘고 사리즈도 적당하다고 하네요 굿입니다


총 606 건 중 7 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 10
2) 리뷰내용: 색상도 사이즈도 너무 마음에 들어요 정말 강추합니다


총 606 건 중 8 번째 리뷰 데이터를 수집합니다====================================
1) 별점: 8
2) 리뷰내용: 자주 신을 것 같습니다
아무 옷이랑 다 잘 어울려요


총 606 건 중 9 번째 리뷰 데이터를 수집합니다================

In [33]:
# 데이터프레임 생성, 각 칼럼 리스트 넣기   
df_style = pd.DataFrame()
df_style['별점'] = score_style
df_style['리뷰내용'] = text_style

# 공백 줄바꿈 제거
for i in range(len(df_style)):
    df_style['리뷰내용'][i] = df_style['리뷰내용'][i].replace("\t","")
    df_style['리뷰내용'][i] = df_style['리뷰내용'][i].replace("\n","")

# csv로 저장
style_csv = search + '/' + result_name + '_style' + '_' + '.csv'
df_style.to_csv(style_csv,encoding="utf-8-sig",index=True)


df_photo = pd.DataFrame()
df_photo['별점'] = score_photo
df_photo['리뷰내용'] = text_photo

# 공백 줄바꿈 제거
for i in range(len(df_photo)):
    df_photo['리뷰내용'][i] = df_photo['리뷰내용'][i].replace("\t","")
    df_photo['리뷰내용'][i] = df_photo['리뷰내용'][i].replace("\n","")

# csv로 저장
photo_csv = search + '/' + result_name + '_photo' + '_' + '.csv'
df_photo.to_csv(photo_csv,encoding="utf-8-sig",index=True)


df_goods = pd.DataFrame()
df_goods['별점'] = score_goods
df_goods['리뷰내용'] = text_goods

# 공백 줄바꿈 제거
for i in range(len(df_goods)):
    df_goods['리뷰내용'][i] = df_goods['리뷰내용'][i].replace("\t","")
    df_goods['리뷰내용'][i] = df_goods['리뷰내용'][i].replace("\n","")

# csv로 저장
goods_csv = search + '/' + result_name + '_goods' + '_' + '.csv'
df_goods.to_csv(goods_csv,encoding="utf-8-sig",index=True)

#data = pd.read_csv(f'{search}/{result_name}_.csv')

<ipython-input-33-5263bf4c417f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_style['리뷰내용'][i] = df_style['리뷰내용'][i].replace("\t","")
<ipython-input-33-5263bf4c417f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_style['리뷰내용'][i] = df_style['리뷰내용'][i].replace("\n","")
<ipython-input-33-5263bf4c417f>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_photo['리뷰내용'][i] = df_photo['리뷰내용'][i].replace("\t","")
<ipython-input-33-

In [34]:
score=score_style+score_photo+score_goods
text=text_style+text_photo+text_goods
df = pd.DataFrame()
df['별점'] = score
df['리뷰내용'] = text

# 공백 줄바꿈 제거
for i in range(len(df_goods)):
    df['리뷰내용'][i] = df['리뷰내용'][i].replace("\t","")
    df['리뷰내용'][i] = df['리뷰내용'][i].replace("\n","")

# csv로 저장
f_csv = search + '/' + result_name + '_최종' + '_' + '.csv'
df.to_csv(f_csv,encoding="utf-8-sig",index=True)

<ipython-input-34-1f92015ca745>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['리뷰내용'][i] = df['리뷰내용'][i].replace("\t","")
<ipython-input-34-1f92015ca745>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['리뷰내용'][i] = df['리뷰내용'][i].replace("\n","")
